##### According to a new study from Princeton University's Woodrow Wilson School,  no matter how much more than $75,000 people make, they don't report any greater degree of happiness (http://content.time.com/time/magazine/article/0,9171,2019628,00.html). 

##### Therefore, in our analysis, we concentrated on metropolitan areas with a median income of $75,000 or more and considered which of these cities would be more affordable based on price per income. 

#### We've reviewed rent, sale prices of homes, income and price per square foot

In [1]:
# Dependencies and Setup
import requests
import json
import codecs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import matplotlib as mpl

In [2]:
# File to Load
price_csv = "Resources/PricePSQ.csv"
affordability_csv = "Resources/affordability_income.csv"
#regionID (used as primary key in other instances) is different in rental_data.csv
rental_csv = "Resources/rental_data.csv"
price_income_csv = "Resources/price_to_income.csv"

In [3]:
# Read Affordability Data
pricing_data = pd.read_csv(price_csv, encoding = "ISO-8859-1")
afford_data = pd.read_csv(affordability_csv, encoding = "ISO-8859-1")
rental_data = pd.read_csv(rental_csv, encoding = "ISO-8859-1")
price_income_data = pd.read_csv(price_income_csv, encoding = "ISO-8859-1")

In [4]:
#Find the cities with median income of 75000 (based on Princeton's study) or greater per family and clean the data 
#This dataframe will only show the income data for December of each year 
#Data shown is from 2010-2018
good_income = afford_data.loc[afford_data["2018-03"] >= 75000, \
              ["RegionID", "RegionName", "2010-03", "2011-03", "2013-03", \
               "2014-03", "2015-03", "2016-03", "2017-03", "2018-03"]]

#Split the column RegionName into two separate columns of RegionName and State
good_income[["RegionName", "State"]]= good_income["RegionName"].str.split(",", expand=True)

#Define a new list called columns_order
columns_order = ["RegionID", "RegionName", "State", "2010-03", "2011-03", "2013-03", \
               "2014-03", "2015-03", "2016-03", "2017-03", "2018-03"]

#Change the order of columns using the columns_order list above
good_income = good_income.reindex(columns=columns_order)

#Rename the column RegionName to City
good_income.rename(columns = {"RegionName" : "City"}, inplace=True)

#Print the top income cities
good_income


,RegionID,City,State,2010-03,2011-03,2013-03,2014-03,2015-03,2016-03,2017-03,2018-03
1,394913,New York,NY,65121.91,64367.99,66087.02,66509.81,68243.81,71139.93,73819.12,77408.08281
7,395209,Washington,DC,86548.90,87700.10,90491.36,90421.65,92518.66,95301.86,98291.83,101670.89510
10,394404,Boston,MA,68693.52,69400.70,73055.94,74598.37,77558.07,81241.06,84234.23,87859.35341
11,395057,San Francisco,CA,72999.45,73138.14,78307.98,81617.52,87056.94,93638.40,99269.76,105089.13310
15,395078,Seattle,WA,63553.48,64249.95,67695.32,69572.76,73824.42,77390.27,80852.92,84056.06621
16,394865,Minneapolis-St Paul,MN,62788.52,63788.84,67128.56,67995.68,70491.18,72750.91,75876.15,78224.95265
17,395056,San Diego,CA,61472.65,61404.55,63239.67,64575.14,67316.70,70619.65,74339.18,78670.95671
20,394358,Baltimore,MD,65119.82,65508.47,68566.60,69698.84,72663.10,74908.19,76904.89,78202.51980
21,394530,Denver,CO,58461.10,59576.35,62815.33,65170.96,68612.92,71833.22,75176.14,78631.51020
34,395059,San Jose,CA,83870.93,85531.04,92039.31,94683.99,100127.20,107486.71,114456.82,121388.69900


### Price to Income for cities where income is greater than 75K

In [5]:
price_to_income = pd.merge(good_income, price_income_data, on='RegionID', how='left')
income = price_to_income[price_to_income['Index'] == 'Price To Income']
march_income = income[["RegionID", "RegionName", "2010_03", "2011_03", "2012_03", "2013_03", "2014_03", "2015_03", "2016_03", "2017_03", "2018_03"]]
march_income


,RegionID,RegionName,2010_03,2011_03,2012_03,2013_03,2014_03,2015_03,2016_03,2017_03,2018_03
0,394913,"New York, NY",5.672438,5.432825,5.220803,5.137166,5.313502,5.314768,5.290981,5.327888,5.465964
3,395209,"Washington, DC",3.700798,3.550737,3.493426,3.639021,3.908356,3.880298,3.839379,3.875195,3.912824
6,394404,"Boston, MA",4.670018,4.479782,4.359354,4.403475,4.659619,4.716466,4.750061,4.982535,5.092413
9,395057,"San Francisco, CA",7.248000,6.733833,6.415946,7.176791,8.099977,8.353154,8.524281,8.564542,8.864445
12,395078,"Seattle, WA",4.733022,4.205451,3.952151,4.090386,4.424145,4.525603,4.831357,5.303457,5.707974
15,394865,"Minneapolis-St Paul, MN",3.092922,2.734021,2.594021,2.721643,2.926657,2.972003,3.062505,3.140644,3.272119
18,395056,"San Diego, CA",6.092140,5.869272,5.656558,6.165434,6.880357,6.907647,7.100007,7.216921,7.414094
21,394358,"Baltimore, MD",3.765367,3.471307,3.269074,3.236270,3.324302,3.238232,3.189237,3.268973,3.349136
24,394530,"Denver, CO",3.823055,3.592029,3.566789,3.741125,3.937337,4.220779,4.598151,4.864575,5.007152
27,395059,"San Jose, CA",6.944003,6.542654,6.414938,7.236039,8.022476,8.462236,8.684795,8.798078,9.993398


In [6]:
rent_to_income = pd.merge(good_income, price_income_data, on='RegionID', how='left')
rent = price_to_income[price_to_income['Index'] == 'Rent Affordability']
march_rent = rent[["RegionID", "RegionName", "2011_03", "2012_03", "2013_03", "2014_03", "2015_03", "2016_03", "2017_03", "2018_03"]]
march_rent

,RegionID,RegionName,2011_03,2012_03,2013_03,2014_03,2015_03,2016_03,2017_03,2018_03
1,394913,"New York, NY",0.363721,0.367102,0.364429,0.386650,0.404256,0.404667,0.385103,0.369670
4,395209,"Washington, DC",0.263945,0.266992,0.265882,0.270068,0.269265,0.265053,0.256135,0.251838
7,394404,"Boston, MA",0.299997,0.312541,0.312090,0.323331,0.326620,0.328652,0.329795,0.320368
11,395057,"San Francisco, CA",0.377696,0.382198,0.368085,0.379624,0.417520,0.426876,0.402056,0.387872
13,395078,"Seattle, WA",0.288000,0.289390,0.281497,0.297013,0.294212,0.303139,0.309748,0.312188
16,394865,"Minneapolis-St Paul, MN",0.251894,0.255770,0.250802,0.258546,0.249904,0.249069,0.246718,0.250415
19,395056,"San Diego, CA",0.392414,0.397406,0.385960,0.398791,0.404298,0.401531,0.394516,0.387446
22,394358,"Baltimore, MD",0.292359,0.291822,0.283345,0.286662,0.279592,0.275377,0.266199,0.266446
25,394530,"Denver, CO",0.280380,0.286072,0.288847,0.304553,0.319881,0.329430,0.317015,0.311884
29,395059,"San Jose, CA",0.334054,0.339878,0.334813,0.344726,0.382673,0.387062,0.359402,0.346277
